# **Detect \#china hate: twitter-roberta-base-hate**

reference: https://huggingface.co/cardiffnlp/twitter-roberta-base-hate

In [65]:
# check which gpu we're using
!nvidia-smi

Fri Mar 18 11:50:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    34W / 250W |   2133MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [66]:
!pip install simpletransformers

In [67]:
# # helper function to save predictions to an output file
# def labels2file(p, outf_path):
# 	with open(outf_path,'w') as outf:
# 		for pi in p:
# 			outf.write(','.join([str(k) for k in pi])+'\n')

In [68]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

## Load Data

In [69]:
import pandas as pd
from collections import Counter

In [70]:
df = pd.read_csv('2021_Q4.csv')
len(df)

241244

In [71]:
df

,author id,created_at,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet
0,49919826,2021-12-31 23:58:03+00:00,1477066552839004160,en,0,0,0,0,Instagram,Happy new year! \n#china #beijing #winter #eve...
1,38971530,2021-12-31 23:57:59+00:00,1477066538331086860,en,0,0,0,0,Paper.li,The latest The Political Science Daily! https:...
2,58010461,2021-12-31 23:57:37+00:00,1477066442705195008,en,0,0,0,0,Twitter Web App,Do you support this? Use your purchasing power...
3,1465236285526880260,2021-12-31 23:57:03+00:00,1477066301612957699,en,0,1,0,0,Twitter for Android,Thoughts?\n\n@beverleyturner @MaajidNawaz\n@Un...
4,783046681,2021-12-31 23:56:45+00:00,1477066228028133380,en,1,0,2,1,Twitter Web App,#Putin Tells #Biden &amp; #EU Cut #NATO Crap &...
...,...,...,...,...,...,...,...,...,...,...
241239,758341474119520256,2021-10-01 00:00:26+00:00,1443727470197346309,en,0,0,0,0,WordPress.com,China's electric carmakers are darting into Eu...
241240,50611261,2021-10-01 00:00:05+00:00,1443727380909023233,en,1,0,0,0,Oktopost,🇨🇳 News story - Argus' #copper concentrate TC/...
241241,3508175477,2021-10-01 00:00:01+00:00,1443727366346485764,en,19,3,2,9,Twitter Web App,Today we mark the 72nd anniversary of the foun...
241242,1219090523811528705,2021-10-01 00:00:00+00:00,1443727360621101072,en,13,4,1,21,Twitter Web App,🇨🇳🎉Today marks the 72nd anniversary of the fou...


In [72]:
tweets = df.tweet

## Preprocess Data

In [73]:
# preprocessing
preprocessed_tweets = []
for tweet in tweets:
  preprocessed_tweet = preprocess(tweet)
  preprocessed_tweets.append(preprocessed_tweet)

## Load Model

In [74]:
from simpletransformers.classification import ClassificationModel

model = ClassificationModel(
    "roberta", "cardiffnlp/twitter-roberta-base-hate")

In [75]:
# get prreds and logits
preds, logits = model.predict(preprocessed_tweets)

  0%|          | 0/241244 [00:00<?, ?it/s]

  0%|          | 0/30156 [00:00<?, ?it/s]

In [76]:
Counter(preds)

Counter({0: 240581, 1: 663})

In [77]:
df['CARDIFFNLP_ROBERTA_PREDS'] = preds
df.to_csv('2021_Q4_1.csv')

In [78]:
df[df['CARDIFFNLP_ROBERTA_PREDS']==1].tweet

218       Urge @MemphisZoo to return #pandas YaYa and Le...
685       PL. NOTE THAT THIS CUNNING/EXPANSIOIST/DEBTORI...
957       #Tigray forces are calling for a ceasefire. Th...
1255      THANK YOU @GREY2KUSA \nVICTORIES FOR THE GREYH...
1756      We should actually thank the #CathayPacific cr...
                                ...                        
238462    4/5 #WALLSTREET #OIL $GAS #FOSSILFUEL #NATURAL...
238669    There were 13 Malays in the flat, why you stil...
239891    we all watched last year when #China  gathered...
240609    Corrupt #China illegal fishing is openly lying...
240734    #BanPakistanChannels 🇵🇰 #Cricket ,don't invite...
Name: tweet, Length: 663, dtype: object

In [79]:
# labels2file([[k] for k in preds], 'twitter-roberta-hate-2021-Q1.txt')

In [80]:
from google.colab import files
files.download('2021_Q2_4.csv')
# files.download('twitter-roberta-hate-2021-Q1.txt')

## _Exp_

In [81]:
# from transformers import AutoModelForSequenceClassification
# from transformers import TFAutoModelForSequenceClassification
# from transformers import AutoTokenizer
# from transformers import Trainer
# import numpy as np
# from scipy.special import softmax
# import csv
# import urllib.request

# # Preprocess text (username and link placeholders)
# def preprocess(text):
#     new_text = []
#     for t in text.split(" "):
#         t = '@user' if t.startswith('@') and len(t) > 1 else t
#         t = 'http' if t.startswith('http') else t
#         new_text.append(t)
#     return " ".join(new_text)

# # Tasks:
# # emoji, emotion, hate, irony, offensive, sentiment
# # stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

# task='hate'
# MODEL = f"cardiffnlp/twitter-roberta-base-hate"

# tokenizer = AutoTokenizer.from_pretrained(MODEL)

# # download label mapping
# labels=[]
# mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/mapping.txt"
# with urllib.request.urlopen(mapping_link) as f:
#     html = f.read().decode('utf-8').split("\n")
#     csvreader = csv.reader(html, delimiter='\t')
# labels = [row[1] for row in csvreader if len(row) > 1]

# # PT
# model = AutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)

In [82]:
# class SimpleDataset:
#     def __init__(self, tokenized_texts):
#         self.tokenized_texts = tokenized_texts
    
#     def __len__(self):
#         return len(self.tokenized_texts["input_ids"])
    
#     def __getitem__(self, idx):
#         return {k: v[idx] for k, v in self.tokenized_texts.items()}

In [83]:
# # get predictions
# preds = []
# count = 0

# preprocessed = []

# with open('twitter-roberta-base-hate-preds.txt', 'a') as preds_file:
#   for tweet in tweets:
#     preprocess(tweet)
#     preprocessed.append(tweet)
  
#     # encoded_input = tokenizer(tweet, return_tensors='pt')
#     # output = model(**encoded_input)
#     # scores = output[0][0].detach().numpy()
#     # scores = softmax(scores)
#     # pred = np.argmax(scores)
#     # preds_file.write(f'{pred}\n')
#     # preds.append(pred)
#     # count += 1
#     # print(f'........................{count}..........................')
#   encoded_inputs = tokenizer(preprocessed, padding=True)
#   # , return_tensors='pt')
#   output = model(**encoded_inputs)
#   test_dataset = SimpleDataset(tokenized_texts)

#   print(output)
# #   scores = output[0][0].detach().numpy()
# #   scores = softmax(scores)
# #   pred = np.argmax(scores)
# #   preds_file.write(f'{pred}\n')
# #   preds.append(pred)
# #   count += 1
# #   print(f'........................{count}..........................')


# # Counter(preds)

In [84]:
# import torch
# from torch.utils.data import TensorDataset, random_split
# from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
# from torch.utils.data import TensorDataset, random_split
# from torch import nn
# from torch.nn import functional as F
# from transformers import BertForSequenceClassification, AdamW, BertConfig
# from transformers import get_linear_schedule_with_warmup
# from transformers import BertTokenizer
# import pandas as pd
# import numpy as np
# import time
# import datetime
# import random
# import glob
# import os
# import sys
# import joblib
# import argparse
# import json
# from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, accuracy_score
# from sklearn import metrics
# from tqdm import tqdm

# USING_GPU = False
# DEVICE = None


# def format_time(seconds):
#     seconds_round = int(round((seconds)))
#     return str(datetime.timedelta(seconds=seconds_round))  # hh:mm:ss


# def prepare_dataset(sentences, labels, tokenizer, max_length=100):
#     input_ids = []
#     attention_masks = []
#     for sent in sentences:
#         # print(sent)
#         try:
#             encoded_dict = tokenizer.encode_plus(
#                 sent,
#                 add_special_tokens=True,
#                 max_length=max_length,
#                 truncation=True,
#                 pad_to_max_length=True,
#                 return_attention_mask=True,
#                 return_tensors='pt'
#             )
#         except:
#             print("some tweet sent is not correct")
#             print(sent)
#             exit(0)

#         input_ids.append(encoded_dict['input_ids'])
#         attention_masks.append(encoded_dict['attention_mask'])

#     input_ids = torch.cat(input_ids, dim=0)
#     attention_masks = torch.cat(attention_masks, dim=0)
#     labels = torch.tensor(labels)
#     return input_ids, attention_masks, labels


# def run_bert_model(model, test_dataset, batch_size, extras=False):
#     print('Predicting labels for {:,} test sentences...'.format(
#         len(test_dataset)))

#     if USING_GPU:
#         print("Using GPU", DEVICE)
#         model.cuda(DEVICE)

#     model.eval()
#     predictions, true_labels = [], []
#     prediction_sampler = SequentialSampler(test_dataset)
#     prediction_dataloader = DataLoader(
#         test_dataset, sampler=prediction_sampler, batch_size=batch_size)
#     for batch in tqdm(prediction_dataloader, total=len(test_dataset)):

#         if extras:
#             with torch.no_grad():
#                 b_proba = model(tokens=batch[0].to(DEVICE),
#                                 masks=batch[1].to(DEVICE),
#                                 extras=batch[3].to(DEVICE))

#                 proba = b_proba.detach().cpu().numpy()
#                 label_ids = batch[2].numpy()

#                 predictions.append(proba)
#                 true_labels.append(label_ids)

#         else:
#             b_input_ids, b_input_mask, b_labels = batch
#             with torch.no_grad():
#                 outputs = model(batch[0].to(DEVICE), token_type_ids=None,
#                                 attention_mask=batch[1].to(DEVICE))
#                 b_proba = outputs[0]

#                 proba = b_proba.detach().cpu().numpy()
#                 label_ids = batch[2].numpy()

#                 predictions.append(proba)
#                 true_labels.append(label_ids)

#     print('    DONE.')

#     flat_predictions = np.concatenate(predictions, axis=0)
#     return flat_predictions

In [85]:
# random.seed(0)
# np.random.seed(0)
# torch.manual_seed(0)
# torch.cuda.manual_seed_all(0)

# global USING_GPU
# global DEVICE
# if torch.cuda.is_available():
#     print('There are %d GPU(s) available.' % torch.cuda.device_count())
#     print('We will use the GPU:', torch.cuda.get_device_name(0))
#     DEVICE = torch.device("cuda:%s" % 0)
#     USING_GPU = True
# else:
#     print('No GPU available, using the CPU instead.')
#     DEVICE = torch.device("cpu")
#     USING_GPU = False

# tag = ""

# # df = pd.read_csv('/content/COVID-HATE-BERT-preds-Q4.csv')
# # df = df[df['tweet'].notna()]
# # X = df.tweet.values

# X = preprocessed_tweets


# input_ids, attention_masks, labels = prepare_dataset(
#     X, np.zeros(len(X)), tokenizer, max_length=400)
# dataset = TensorDataset(input_ids, attention_masks, labels)

# y_pred = np.zeros(len(X))

# flat_logits = run_bert_model(
#     model, dataset, batch_size=8, extras=False)
# y_pred = np.argmax(flat_logits, axis=1).flatten()
# print(y_pred)

# # df['cardiffnlp-RoBERTa-preds'] = y_pred
# # df.to_csv('whatwhat.csv')

In [86]:
# Counter(y_pred)

In [87]:
# !cat twitter-roberta-base-hate-preds.txt

In [88]:
# labels2file([[k] for k in preds], 'twitter-roberta-base-hate-Q1.txt')

In [89]:
# from google.colab import files
# files.download('twitter-roberta-base-hate-Q1.txt')